In [278]:
import pandas as pd
from nltk.cluster.util import cosine_distance
import numpy as np
from flask import Flask, request
application = Flask(__name__)

In [279]:
#Reading the input file
def filereader(file):
    statements=[]
    reader =open(file, 'r')
    for line in reader:
        statements.append(line)

    for line in range(0,len(statements)):
        statements[line]=statements[line].replace("\r\n", "").split(" ") #removing unused tags in the dataset
    return statements

In [280]:
def sentenceSimilarityScore(sentA, sentB):
    all_words = list(set(sentA+sentB))
    vecA = [0]*len(all_words)
    vecB = [0]*len(all_words)
    for w in sentA:
        vecA[all_words.index(w)]+=1
    for w in sentB:
        vecB[all_words.index(w)]+=1
    return 1-cosine_distance(vecA, vecB)


In [281]:
def SimilarityMatrix(statements):
    sim_matrix = np.zeros((len(statements),len(statements)))
    #verify above
    for i in range(len(statements)):
        for j in range(len(statements)):
            if i==j:
                continue
            sim_matrix[i][j] = sentenceSimilarityScore(statements[i], statements[j])
    return sim_matrix


In [282]:
def MatrixtoStatements(soltuple, statements):
    output =[]
    for i in range(len(soltuple)):
        if (soltuple[i] == -2):
            output.append(statements[i])
            continue
        if (soltuple[i] == 0):
            continue
        ans = statements[i], statements[soltuple[i]]
        output.append(ans)
        return output

In [283]:
@application.route('/semantic/similarity',methods=['POST'])
def SemanticSimilarity():
    statement = request.get_json()
    statements = statement['name']
    for line in range(0,len(statements)):
        statements[line]=statements[line].replace("\r\n", "").split(" ") 
    sim_matrix = SimilarityMatrix(statements)
    soltuple=[0]*len(statements)
    visited=np.zeros(len(statements))
    for i in range (0,len(statements)):
        max=0;
        if(visited[i]!=0):
            continue
        visited[i]=1
        occurence = -2
        for j in range(i,len(statements)):
            if(visited[j]!=0):
                continue
            if (sim_matrix[i][j]>max):
                max = sim_matrix[i][j]
                occurence = j
        soltuple[i]=occurence
        if(occurence!=-2):
            visited[occurence]=1   
    answer = MatrixtoStatements(soltuple, statements)
    response = ''.join(str(a) for a in answer)
    return response


In [ ]:
application.run(host = '0.0.0.0')